In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.coordinates as coord
import astropy.units as u
from astropy import time

from poliastro.bodies import Earth
from poliastro.twobody import Orbit
import poliastro

import urllib.request
import json

In [2]:
url = 'http://www.celestrak.com/NORAD/elements/stations.txt'


response = urllib.request.urlopen(url)
data = response.read()      # a `bytes` object
text = data.decode('utf-8') # a `str`; this step can't be used if data is binary
data = text.splitlines()

for i in range(len(data)):
    if data[i].startswith('ISS (ZARYA)'):
        
        l1 = "1 25544U 98067A   19190.67737078  .00000405  00000-0  14728-4 0  9990"
        l2 = "2 25544  51.6431 249.6881 0007257 122.7057  10.0638 15.50965501178747"
        
        l1 = data[i+1].split()
        l2 = data[i+2].split()
        
        
        time_vector = l1[3]        
        year = int("20"+time_vector[0:2])
        ydayfraction = float(time_vector[2:])
        
                
        Inclination    = float(l2[2]) * u.deg
        RA_of_node     = float(l2[3]) * u.deg
        Eccentricity   = float("0."+l2[4]) * u.one
        Arg_of_perigee = float(l2[5]) * u.deg
        Mean_anomaly   = float(l2[6]) * u.deg
        Rev_Per_Day    = (float(l2[7])  / u.day).to(1/u.s)
        semi_major_Axis = Earth.k**(1/3) / (2 * np.pi * Rev_Per_Day)**(2/3)
        

        print("ISS TLE from celestrack.")
        print(data[i+1])
        print(data[i+2])
        print("\n")
        
        message = ("\tInclination    : {}\n"
                   "\tRA of node     : {}\n"
                   "\tEccentricity   : {}\n"
                   "\tArg of Perigee : {}\n"
                   "\tMean_Anomaly   : {}\n"
                   "\tSemi-major Axis: {}").format(Inclination, 
                                                   RA_of_node, 
                                                   Eccentricity,
                                                   Arg_of_perigee, 
                                                   Mean_anomaly, 
                                                   semi_major_Axis)
    
        print("ISS Orbital Elements from TLE\n\n" + message ) 
              
            
Epoch_time = time.Time("{}-01-01 00:00".format(year))
Epoch_time += (ydayfraction -1) * u.day

print("TLE at: ", Epoch_time.yday, " that means", Epoch_time)

ZAYRA = poliastro.twobody.Orbit.from_classical(Earth,
                                              semi_major_Axis,
                                              Eccentricity,
                                              Inclination,
                                              RA_of_node,
                                              Arg_of_perigee,
                                              Mean_anomaly,
                                              Epoch_time)

Tinit = time.Time("2019-07-11 2:03")
DeltaT = Tinit - Epoch_time
#print(Tnow,"\n", Epoch_time,"\n", DeltaT)

ISSnow = ZAYRA.propagate( DeltaT )

ISS TLE from celestrack.
1 25544U 98067A   19191.64362958  .00000242  00000-0  11957-4 0  9999
2 25544  51.6430 244.8938 0007211 126.3371   5.0810 15.50966263178893


ISS Orbital Elements from TLE

	Inclination    : 51.643 deg
	RA of node     : 244.8938 deg
	Eccentricity   : 0.0007211
	Arg of Perigee : 126.3371 deg
	Mean_Anomaly   : 5.081 deg
	Semi-major Axis: 6792040.610266549 m
TLE at:  2019:191:15:26:49.596  that means 2019-07-10 15:26:49.596


The state of ISS (r,v) in the Non Rotating Frame is:

In [3]:
ISSnow.r

<Quantity [ 3136.22352362, -3020.85395588,  5208.26903375] km>

In [4]:
ISSnow.v

<Quantity [4.07088181, 6.37431937, 1.24046992] km / s>

Now we load the ground stations info.

In [5]:
with open('ground_stations.json') as json_file:
    data = json.load(json_file)

In [6]:
GS = []
gs = []

for  k in sorted(data.keys()):
    GS.append(data[k])

In [7]:
for k in GS:
    gs.append(coord.EarthLocation( k["gs_lon"], k["gs_lat"], k["gs_alt"]))
    

Their location in the rotating system is:

In [8]:
gs

[<EarthLocation (3946293.3013581, 809282.50717543, 4928547.87338264) m>,
 <EarthLocation (4157409.29535264, 672080.10355092, 4774350.40490151) m>,
 <EarthLocation (3899925.15472251, 855534.85353504, 4957331.36135458) m>,
 <EarthLocation (3857116.33132939, 516634.17159883, 5036460.26932419) m>]

In [9]:
gs[0].to_geodetic()

GeodeticLocation(lon=<Longitude 11.5892 deg>, lat=<Latitude 50.9271 deg>, height=<Quantity 143. m>)

In [10]:
gs_gcrs = []
gs_vel_gcrs = []

In [11]:
for k in gs:
    gs_gcrs.append( k.get_itrs(Tinit) )
    gs_vel_gcrs.append( k.get_gcrs_posvel(obstime= Tinit))

In [12]:
gs_vel_gcrs

[(<CartesianRepresentation (x, y, z) in m
      (3525405.75533251, -1965897.04909039, 4921942.31878877)>,
  <CartesianRepresentation (x, y, z) in m / s
      (143.348391, 256.40649168, -0.26263015)>),
 (<CartesianRepresentation (x, y, z) in m
      (3595291.20334913, -2207700.45892648, 4767610.02492607)>,
  <CartesianRepresentation (x, y, z) in m / s
      (160.98116313, 261.52360268, -0.29543932)>),
 (<CartesianRepresentation (x, y, z) in m
      (3520578.43009277, -1900587.0501085, 4950735.97626316)>,
  <CartesianRepresentation (x, y, z) in m / s
      (138.5859254, 256.05053449, -0.25375251)>),
 (<CartesianRepresentation (x, y, z) in m
      (3266885.76846163, -2129173.4179394, 5030334.70430598)>,
  <CartesianRepresentation (x, y, z) in m / s
      (155.25449902, 237.54015805, -0.28520568)>)]

In [13]:
a = gs[0].get_gcrs_posvel(obstime=Tinit)[0]

In [14]:
a.xyz

<Quantity [ 3525405.75533251, -1965897.04909039,  4921942.31878877] m>

In [15]:
ISSnow.r - a.xyz

<Quantity [ -389.18223171, -1054.95690679,   286.32671496] km>

In [16]:
ISSnow.v - gs[0].get_gcrs_posvel(obstime=Tinit)[1].xyz

<Quantity [3.92753342, 6.11791288, 1.24073255] km / s>

In [17]:
ISSnow.v

<Quantity [4.07088181, 6.37431937, 1.24046992] km / s>

In [18]:
gs[0].get_gcrs_posvel(obstime=Tinit)[1].xyz

<Quantity [143.348391  , 256.40649168,  -0.26263015] m / s>

In [19]:
R_iss_gs0 = ISSnow.r - a.xyz

In [20]:
R_iss_gs0

<Quantity [ -389.18223171, -1054.95690679,   286.32671496] km>

In [21]:
dist = 0
for k in R_iss_gs0:
    dist += k**2
dist = (dist)**0.5

In [22]:
dist

<Quantity 1160.33610319 km>

In [23]:
R_iss_gs0 / dist

<Quantity [-0.33540474, -0.90918218,  0.24676188]>

In [24]:
R_iss_gs0_unitary = R_iss_gs0 / dist

In [25]:
V_iss_gs0 = ISSnow.v - gs[0].get_gcrs_posvel(obstime=Tinit)[1].xyz

In [26]:
V_iss_gs0.dot(R_iss_gs0_unitary)

<Quantity -6.5734452 km / s>

Ok,

We have the ISS close to Germany on Thursday 2019-07-11 at 2:05 GMT

According to https://spotthestation.nasa.gov/ the station will pass over germany in 5 minutes.

The time window will be set for 8 minutes, sampling each 10 seconds.

For testing purposes, all stations are sampling synchronously.

In [27]:
Tinit

<Time object: scale='utc' format='iso' value=2019-07-11 02:03:00.000>

In [28]:
deltat = 10 * u.s

In [29]:
ISS_evolution = []
T = []
GS0_loc = []
GS1_loc = []
GS2_loc = []
GS3_loc = []


for i in range(30):
    T.append(Tinit + deltat * i)
    ISS_evolution.append( ISSnow.propagate(deltat * i) )
    GS0_loc.append( gs[0].get_gcrs_posvel(obstime = Tinit + deltat*i) )
    GS1_loc.append( gs[1].get_gcrs_posvel(obstime = Tinit + deltat*i) )
    GS2_loc.append( gs[2].get_gcrs_posvel(obstime = Tinit + deltat*i) )
    GS3_loc.append( gs[3].get_gcrs_posvel(obstime = Tinit + deltat*i) )

In [30]:
ISSnow.r

<Quantity [ 3136.22352362, -3020.85395588,  5208.26903375] km>

In [31]:
def norm( v ):
    norm = 0
    for k in v:
        norm += k**2
    norm = norm**0.5
    
    return norm

# Data for JENA

In [32]:
for o, gs, t in zip(ISS_evolution, GS0_loc, T):
    iss_rel_to_gs = o.r - gs[0].xyz
    V_iss_rel_to_gs = o.v - gs[1].xyz
    V = V_iss_rel_to_gs
    
    dist = norm(iss_rel_to_gs)
    
    iss_rel_to_gs_unitary = iss_rel_to_gs / dist
    u = iss_rel_to_gs_unitary
    
    
    
    print(t,dist, V.dot(u))

2019-07-11 02:03:00.000 1160.3361031856693 km -6.573445202693996 km / s
2019-07-11 02:03:10.000 1094.9127710278253 km -6.509229518083981 km / s
2019-07-11 02:03:20.000 1030.196072618725 km -6.431619142630671 km / s
2019-07-11 02:03:30.000 966.3364056692445 km -6.337167271568823 km / s
2019-07-11 02:03:40.000 903.5238585825659 km -6.221326915861143 km / s
2019-07-11 02:03:50.000 842.0010660696095 km -6.078060466185827 km / s
2019-07-11 02:04:00.000 782.0806039009605 km -5.899318863171794 km / s
2019-07-11 02:04:10.000 724.1683046629236 km -5.674377912609743 km / s
2019-07-11 02:04:20.000 668.7937967051495 km -5.389072102079201 km / s
2019-07-11 02:04:30.000 616.6485819669922 km -5.0251024531055855 km / s
2019-07-11 02:04:40.000 568.6288036897915 km -4.559910994697591 km / s
2019-07-11 02:04:50.000 525.8722978414614 km -3.9681891518313144 km / s
2019-07-11 02:05:00.000 489.7652096528488 km -3.2267985910212564 km / s
2019-07-11 02:05:10.000 461.87444624466644 km -2.3248633845025823 km / s

In [33]:
GroundStationLocations = [GS0_loc, GS1_loc, GS2_loc, GS3_loc]

In [34]:
for GS_loc in GroundStationLocations:
    for o, gs, t in zip(ISS_evolution, GS_loc, T):
        iss_rel_to_gs = o.r - gs[0].xyz
        V_iss_rel_to_gs = o.v - gs[1].xyz
        V = V_iss_rel_to_gs

        dist = norm(iss_rel_to_gs)

        iss_rel_to_gs_unitary = iss_rel_to_gs / dist
        u = iss_rel_to_gs_unitary



        print(t,dist, V.dot(u))
        
    print("\n\n")

2019-07-11 02:03:00.000 1160.3361031856693 km -6.573445202693996 km / s
2019-07-11 02:03:10.000 1094.9127710278253 km -6.509229518083981 km / s
2019-07-11 02:03:20.000 1030.196072618725 km -6.431619142630671 km / s
2019-07-11 02:03:30.000 966.3364056692445 km -6.337167271568823 km / s
2019-07-11 02:03:40.000 903.5238585825659 km -6.221326915861143 km / s
2019-07-11 02:03:50.000 842.0010660696095 km -6.078060466185827 km / s
2019-07-11 02:04:00.000 782.0806039009605 km -5.899318863171794 km / s
2019-07-11 02:04:10.000 724.1683046629236 km -5.674377912609743 km / s
2019-07-11 02:04:20.000 668.7937967051495 km -5.389072102079201 km / s
2019-07-11 02:04:30.000 616.6485819669922 km -5.0251024531055855 km / s
2019-07-11 02:04:40.000 568.6288036897915 km -4.559910994697591 km / s
2019-07-11 02:04:50.000 525.8722978414614 km -3.9681891518313144 km / s
2019-07-11 02:05:00.000 489.7652096528488 km -3.2267985910212564 km / s
2019-07-11 02:05:10.000 461.87444624466644 km -2.3248633845025823 km / s